# validate t_part

compare t_part with [Kangxi 214 radicals](https://www.xiaoma.info/bushou.php?ext=1)

In [1]:
from utils_copy import *

from IPython.display import Image

In [37]:
sql_stmt = f"""
select --zi, 
case when --coalesce(alias,traditional) is not null  and 
	coalesce(alias,traditional) != '' then 
	zi || '(' || coalesce(alias,traditional) || ')' 
	ELSE zi
end as Radicaal, 
strokes, pinyin, meaning as definition 
from t_part 
order by cast(strokes as int), 
zi;
"""
with DBConn() as _conn:
    df1 = pd.read_sql(sql_stmt, _conn)

df1 

,Radicaal,strokes,pinyin,definition
0,丬,None,None,None
1,一,1,yī,one
2,丨,1,shù,vertical line Through Alone Straight
3,丶,1,zhǔ,dot / point
4,丿(乀乁),1,piě,slash
...,...,...,...,...
409,畺,13,jiāng,None
410,鼓,13,gǔ,drum
411,鼠,13,shǔ,rat
412,鼻,14,bí,nose


In [38]:
df1.to_csv('t_part_radicals.csv', index=False)

## read table from HTML

below code is provided by Claude 3.5 sonnet on 2024-07-07

```
Using python, How can one read and parse the HTML table from this URL : "https://www.xiaoma.info/bushou.php?ext=1" and save it to pandas dataframe
```

In [29]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# # Send a GET request to the URL
# url = "https://www.xiaoma.info/bushou.php?ext=1"
# response = requests.get(url)

# Parse the HTML content
# soup = BeautifulSoup(response.content, 'html.parser')

## because the page resist scraping, save the page manually to a local HTML file

# Read the local HTML file
with open('Xiaoma Cidian - Kangxi Radicals.html', 'r', encoding='utf-8') as file:
    html_content = file.read()


# Parse the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# Find the table in the HTML
table = soup.find('table', {'class': 'radicals'})

trows = table.find_all('tr')

headers = [td.text.strip() for td in trows[0].find_all('td')]

# Extract table rows
rows = []
for tr in trows[1:]:  # Skip the header row
    row = [td.text.strip() for td in tr.find_all('td')]
    rows.append(row)

# Create DataFrame
df = pd.DataFrame(rows, columns=headers)

df = df[df.columns[1:]]  # skip index column

## df.head()

# Save the DataFrame to a CSV file (optional)
df.to_csv('kanxi_radicals.csv', index=False)

In [36]:
df

,Radical,Strokes,Pinyin,Definition
0,一,1,yī,one
1,丨,1,gǔn,line
2,丶,1,zhǔ,dot
3,丿 (乀乁),1,piě,slash
4,乙 (乚乛),1,yǐ,second
...,...,...,...,...
209,齊 (齐),14,qí,neat
210,齒 (齿),15,chǐ,teeth
211,龍 (龙),16,lóng,dragon
212,龜 (龟),16,guī,turtle
